# HW6 Edge Detection: Canny

ECE472: Replace the homegrown Gaussian smoothing kernel and Sobel gradient kernel with the similar skimage filters. Determine settings for sigma, contrast rescaling, low and high tresholds that produce a visually satifying result for the umbc.png building image. Define the contrast scaling as a fraction of the largest gradient magnitude. Call this image C1. Using the same sigma and contrast rescaling settings, show what happens when the low and high thresholds are doubled. Call this image C2. Display C1 and C2 side-by-side with titles that include the parameter settings. Add a brief observations statement.

Hint: You need to compute horizontal and vertical gradients. By default, the skimage Sobel filter returns the gradient magnitude from which you cannot compute gradient angles.

ECE572: Complete the ECE472 assignment. Verify/correct the Canny support functions for angle quantization, edge thinning, and edge hysteresis.

Bragging rights: Determine if the given Sobel gradient kernel produces the same output as the skimage Sobel filter and if not, what makes them different. Introduce interpolation based edge thinning.

In [ ]:
%load_ext cython

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from skimage import io
from skimage import exposure

from skimage.util import img_as_float32 as img_as_float

In [ ]:
def print_imginfo(I):
    # Copy from HW4

In [ ]:
def show_images(I, titles=None):
    # Copy from HW5 -- since it was optional then, you may have to write it now 

In [ ]:
# read umbc.png image

## ECE472: SKImage Smoothing, Sobel Gradients

In [ ]:
# Apply skimage based Gaussian smooothing (sigma)
# Compute horizonal and vertical Sobel gradients
# Combine to form complex image
# Compute gradient magnitudes
# Rescale using fraction of max gradient magnitude
# Result should be complex image called I2

## ECE572: Verify/Correct Edge Thinning, Hysteresis Code

In [ ]:
%%cython 
import numpy as np

# Quantize into four gradient angles
# Note: np.angle(x,y) = atan2(y,x)
#       angle range [-180,180] deg
#
# Q0:   0 deg:  (x+1,y),   (x-1,y)
# Q1:  45 deg:  (x+1,y+1), (x-1,y-1)
# Q2:  90 deg:  (x,y+1),   (x,y-1)
# Q3: 135 deg:  (x+1,y-1), (x-1,y+1)

cpdef angle_quantization(Ga):
    Q = np.around(((Ga+180)%180)/45)
    return Q%4

# Apply non-max gradient suppression

cpdef edge_thinning(Gm, Qa):
    Qm = np.copy(Gm)
    Gz = np.pad(Gm, ((1,1),(1,1)), mode='constant', constant_values=0)
    
    cdef int u, v
    cdef float Muv, M1, M2
    
    for u in np.arange(0,Gm.shape[0]):
        for v in np.arange(0,Gm.shape[1]):
            Muv = Gm[u][v]
            
            if Qa[u][v] == 0:
                M1 = Gz[u+0][v+1]
                M2 = Gz[u+2][v+1]
                   
            elif Qa[u][v] == 1:
                M1 = Gz[u+2][v+2]
                M2 = Gz[u+0][v+0]
                     
            elif Qa[u][v] == 2:
                M1 = Gz[u+1][v+2]
                M2 = Gz[u+1][v+0]
                
            elif Qa[u][v] == 3:
                M1 = Gz[u+2][v+0]
                M2 = Gz[u+0][v+2]
                
            if Muv < M1 or Muv < M2:
                Qm[u][v] = 0.0
                
    return Qm

cpdef edge_tracing(int u0, int v0, E, Qm, float low):
    cdef int u1, u2, u
    cdef int v1, v2, v
 
    u1 = u0 if u0 == 0 else u0-1
    u2 = u0 if u0 == E.shape[0]-1 else u0+1
    v1 = v0 if v0 == 0 else v0-1
    v2 = v0 if v0 == E.shape[1]-1 else v0+1
    
    E[u0][v0] = Qm[u0][v0]
    
    for u in np.arange(u1,u2+1):
        for v in np.arange(v1,v2+1):
            if E[u][v] == 0.0 and Qm[u][v] >= low:
                edge_tracing(u, v, E, Qm, low)
                return
    return
    
cpdef edge_hysteresis(Qm, float low=0.10, float high=0.20, threshold=True):
    E = np.zeros_like(Qm)
    
    cdef int u, v
    
    for u in np.arange(0,Qm.shape[0]):
        for v in np.arange(0,Qm.shape[1]):            
            if E[u][v] == 0.0 and Qm[u][v] >= high:
                edge_tracing(u, v, E, Qm, low)
    
    if threshold:
        E = np.where(0<E, 1.0, 0.0)
        
    return E

## ECE472: Threshold Parameter Settings

In [ ]:
Ga = np.angle(I2, deg=True)
Qa = angle_quantization(Ga)

Qm = edge_thinning(Gm, Qa)

Display histogram of quantized gradients

low = [?, 2x]
high = [?, 2x]

C1 = edge_hysteresis(Qm, low=low[0], high=high[0], threshold=True)
C2 = edge_hysteresis(Qm, low=low[1], high=high[1], threshold=True)

In [ ]:
# Binarize Canny edge maps
# Display result w/sigma, contrast scaling, and hysteresis thresholds listed in title

## Observations